# WALLABY user download notebook

A notebook pre-filled with cells and scripts for downloading WALLABY internal release data. The notebook has the following sections

1. Connect to the database
2. Get source finding catalog and products
3. Get kinematic model table and products

<span style="font-weight: bold; color: #FF0000;">⚠ Make sure the Jupyter Notebook server is loaded with the wallaby/python-3.9.1 module!</span>

<span style="font-weight: bold; color: #FF0000;">⚠ If the Jupyter Notebook server is not loaded with the wallaby/python-3.9.1 delete the interactive session and start a new one with the correct module.</span>

---

# 1. Connect

Establish a connection to the database with the `wallaby_data_access` module, available on PyPI [https://pypi.org/project/wallaby-data-access/](https://pypi.org/project/wallaby-data-access/).

In [ ]:
# relevant libraries

import os
import wallaby_data_access as wallaby
from astropy.table import Table

In [ ]:
# connect to the database

wallaby.connect()

# 2. Source finding

The intention for this notebook is to allow users to easily download all of the products and properties for all sources of a given tag. We start by listing the available tags in the cell below.

In [ ]:
# Get all tags

wallaby.print_tags()

In [ ]:
# Select the tag

tag_name = "NGC 5044 DR1"

In [ ]:
# Retrieve catalogue as Astropy table

table = wallaby.get_catalog(tag_name)

# Sort table by flux (brightest first)
table.sort("f_sum", reverse=True)

# Print table
table.pprint()

## Download catalog

It is convenient to write `astropy` Table objects to a number of file formats. The documentation for `write` can be found [here](https://docs.astropy.org/en/stable/io/ascii/write.html). Below we show how to export the table as a `.fits` file. 

<span style="font-weight: bold; color: #FF0000;">⚠ You cannot write a table with column of type "object".</span> In this example we are replacing the object columns with strings containing the same information.

In [ ]:
# Write table to fits file

catalog_filename = "%s_catalog.fits" % tag_name.replace(' ', '_')
write_table = table

if not os.path.exists(catalog_filename):
    # Update write table comment and tag fields
    write_table['comments'] = [''.join(comment) for comment in write_table['comments']]
    write_table['tags'] = [''.join(t) for t in write_table['tags']]
    write_table.write(catalog_filename, format = 'fits')

## Download products

We can store the source finding output products for a source, or the entire release. In the cells below we show how to do this.

In [ ]:
# Write output products for a source

row = table[0]
id = int(row['id'])
name = row['name'].replace(' ', '_')
wallaby.save_products_for_source(id, f'{name}_source_products')

In [ ]:
# Write all output products for tagged sources (only selecting a subset)

wallaby.save_source_products(table[0:5], f"{tag_name.replace(' ', '_')}_source_products")

---

# Kinematic models

This notebook also allows users to download the kinematic model table and products. First, we select the kinematic models by tag and retrieve the table.

In [ ]:
wallaby.get_kinematic_model_tags()

In [ ]:
kin_tag = 'NGC 5044 Kin TR1'

In [ ]:
kin_table = wallaby.get_kinematic_model(team_release_kin=kin_tag)
kin_table[0:2]

Then we can write the table as a `.fits` file and all products associated with the selected models. As with the source finding table, it will be necessary to replace the object columns with strings.

In [ ]:
# Write table

string_array_columns = ['rad', 'vrot_model', 'e_vrot_model', 'e_vrot_model_inc', 'rad_sd', 'sd_model', 'sd_fo_model', 'e_sd_model', 'e_sd_fo_model_inc']
catalog_filename = "%s_catalog.fits" % kin_tag.replace(' ', '_')
write_table = kin_table

# Convert object objects to strings
for column in string_array_columns:
    array = write_table[column]
    string_array = []
    for elem in array:
        string_elem = ','.join([str(v) for v in list(elem)])
        string_array.append(string_elem)
    write_table[column] = string_array

if not os.path.exists(catalog_filename):
    kin_table.write(catalog_filename, format = 'fits')

In [ ]:
# Save single product file

row = kin_table[0]
id = int(row['id'])
name = str(row['source']).replace(' ', '_')
wallaby.save_products_for_kinematic_model(id, f'{name}_kinematic_products')

In [ ]:
# Save products for all kinematic models

wallaby.save_kinematic_model_products(kin_table[0:2], f"{kin_tag.replace(' ', '_')}_kin_products")